<a href="https://colab.research.google.com/github/Hendyley/Pole_Dancer/blob/main/SC3000_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h1><center>**SC3000 Assignment 1 Balancing a Pole on a Cart**<center></h1>

---


<center><img src="https://miro.medium.com/max/694/1*iOceBp5ks4X_Bs-8vqT2qA.png" href="https://miro.medium.com/max/694/1*iOceBp5ks4X_Bs-8vqT2qA.png"></center>


##### Source: https://miro.medium.com/max/694/1*iOceBp5ks4X_Bs-8vqT2qA.png

<hr></hr>

<h2><center><strong>Lab Group : A42<strong><center></h2>
<h2><center>Group Name: <strong>PoleDancer</strong><center></h2>
<ul>
<li>Brendon Tan : Task 1, 2, 3, 4</li>
<li>Hendy : Task 1, 2, 3, 4</li>
<li>Wong Ri Hong : Task 1, 2, 3, 4</li>
</ul>

<hr></hr>


Reference: https://medium.com/swlh/using-q-learning-for-openais-cartpole-v1-4a216ef237df


### Installing dependencies:

> Indented block



###### Reference:
###### Python Gymnasium source: https://gymnasium.farama.org/


In [8]:
!pip install requests
!pip install --upgrade setuptools
!pip install scores
!pip install gymnasium
!pip install Box2D
!pip install tools

  Using cached Box2D-2.3.2.tar.gz (427 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for Box2D: started
  Running setup.py install for Box2D: finished with status 'error'


  DEPRECATION: Box2D is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  Running setup.py install for Box2D did not run successfully.
  exit code: 1
  
  [18 lines of output]
  Using setuptools (version 67.6.1).
  running install
  C:\Users\Hendy\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
    warnings.warn(
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\Box2D
  copying library\Box2D\Box2D.py -> build\lib.win-amd64-cpython-311\Bo

In [9]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install gym[classic_control]
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

The system cannot find the path specified.


The system cannot find the path specified.


  Using cached pygame-2.1.0.tar.gz (5.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [77 lines of output]
  
  
  WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
  Using WINDOWS configuration...
  
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Hendy\AppData\Local\Temp\pip-install-dj3bwy1g\pygame_e35891ef1c6d4c348203ef0e1009d774\setup.py", line 388, in <module>
      buildconfig.config.main(AUTO_CONFIG)
    File "C:\Users\Hendy\AppData\Local\Temp\pip-install-dj3bwy1g\pygame_e35891ef1c6d4c348203ef0e1009d774\buildconfig\config.py", line 234, in main
      deps = CFG.main(**kwds)
             ^^^^^^^^^^^^^^^^
    File "C:\Users\Hendy\AppData\Local\Temp\pip-install-dj3bwy1g\pygame_e35891ef1c6d4c348203ef0e1009d774\buildconfig\config_win.py", line 511, in main
      return setup_prebuilt_sdl2(prebuilt_dir)
             ^^^^^^^^^^^^^^^^^^^^^^

The system cannot find the path specified.


## Importing dependencies and define helper functions

In [10]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
from gym.wrappers.monitoring import video_recorder
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import os
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [11]:
!pip install matplotlib

## Loading CartPole environment

In [12]:
env = gym.make("CartPole-v1")

We can check the action and observation space of this environment. Discrete(2) means that there are two valid discrete actions: 0 & 1.

In [13]:
print(env.action_space)

Discrete(2)


The observation space is given below. The first two arrays define the min and max values of the 4 observed values, corresponding to cart position, velocity and pole angle, angular velocity.

In [14]:
print(env.observation_space)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


We call each round of the pole-balancing game an "episode". At the start of each episode, make sure the environment is reset, which chooses a random initial state, e.g., pole slightly tilted to the right. This initialization can be achieved by the code below, which returns the observation of the initial state.

In [15]:
observation = env.reset()
print("Initial observations:", observation)


print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Initial observations: (array([-0.03510504,  0.0276076 ,  0.01148383,  0.01843208], dtype=float32), {})
Action Space Discrete(2)
State Space Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


For the CartPole environment, there are two possible actions: 0 for pushing to the left and 1 for pushing to the right. For example, we can push the cart to the left using code below, which returns the new observation, the current reward, an indicator of whether the game ends, and some additional information (not used in this project). For CartPole, the game ends when the pole is significantly tilted or you manage to balance the pole for 500 steps. You get exactly 1 reward for each step before the game ends (i.e., max cumulative reward is 500).

In [16]:
observation, reward, done, info = env.step(0)
#observation, reward, done, info, terminated, truncated = env.step(0)
print("New observations after choosing action 0:", observation)
print("Reward for this step:", reward)
print("Is this round done?", done)

C:\Users\Hendy\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ValueError: too many values to unpack (expected 4)

Now we can play a full round of the game using a naive strategy (always choosing action 0), and show the cumulative reward in the round. Note that reward returned by env.step(*) corresponds to the reward for current step. So we have to accumulate the reward for each step. Clearly, the naive strategy performs poorly by surviving only a dozen of steps.

In [ ]:
env = gym.make("CartPole-v1")
observation = env.reset()
cumulative_reward = 0
done = False
while not done:
    observation, reward, done, info = env.step(0)
    cumulative_reward += reward
print("Cumulative reward for this round:", cumulative_reward)

## Task 1: Development of an RL agent

An example of a naive agent is given below, which randomly chooses an action regardless of the observation:

In [ ]:
def rand_policy_agent(observation):
    return random.randint(0, 1)

## Initialize parameters for Q-learning

In [ ]:
LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 40000
total_reward = 0
prior_reward = 0

observation = env.reset()
print("Initial observations:", observation)
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))


Observation = [30, 30, 100, 100]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])
epsilon = 1
epsilon_decay_value = 0.9999

q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))

def get_discrete_state(state):
    discrete_state = state/np_array_win_size+ np.array([15,15,50,50])
    return tuple(discrete_state.astype(int))

This is our Q-learning model. 
We set our learning rate to be low at 0.1 because we do not want the agent to overreact to new information which will cause it to overwrite its old information. 
Our discount rate is 0.95 to accommodate long term rewards.
We determine the number of episodes of 40000 to be sufficient for the exploration (not too low/high) and exploitation for the Q-learning agent.
We initialize total reward and prior reward to 0.

We set the observation space to be [30, 30, 100, 100] as the first 2 variables (Cartpole position and cartpole velocity) are not as important as the next 2 variables (Pole Angle and pole velocity). It is unlikely that the cartpole will end up at the extreme ends of the boundary (edge of the display) and the velocity is bounded by a small range. Whereas for the pole, when it topples, it will definitely reach the extreme angles.

Similarly, we set the window size to be [0.25, 0.25, 0.01, 0.1] for a similar reason, for more precision on pole angle and velocity as these 2 factors are more important then cartpole position and velocity.

Initialisation of epsilon to 1 is because there is no information to exploit initially.
We determine that the epsilon decay value of 0.9999 is sufficient for our model’s exploitation.

We initialize the Q-table with random values (0 or 1) for each state-action pair in the environment since the agent has no knowledge initially of which action to take to maximize the reward.

We define a function get_discrete_state to classify each state into a discrete state based on which window it fits into. We add [15,15,50,50] to the discrete array to avoid negative indexing in our Q-table.



## Q learning, train the agent on 40000 episodes

In [ ]:
for episode in range(EPISODES + 1): 
    
    #Initializing starting state for each epsiode in the given environment range
     
    discrete_state = get_discrete_state(env.reset())   
    done = False
    episode_reward = 0 

    if episode % 2000 == 0: 
        print("Episode: " + str(episode))

    while not done: 

        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state]) # exploit
        else:

            action = np.random.randint(0, env.action_space.n) #explore

        new_state, reward, done, info = env.step(action) #get new state and reward after taking the action above
        episode_reward += reward # add reward to current episode total reward
        new_discrete_state = get_discrete_state(new_state) # discretize the new state
           
        if not done: 
            max_future_q = np.max(q_table[new_discrete_state]) #get max reward from q table
            
            # Using bellman equation to calculate new q-value
            
            current_q = q_table[discrete_state + (action,)] 

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state # Replace current state with the new discrete state

    if epsilon > 0.05:  # Prevent epsilon from dropping too low, so that the model will keep exploring even at high episodes
        if episode_reward > prior_reward and episode > 10000: # If episode reward is greater than previous episode reward, it means that our model is improving, so we should exploit more
                                                              #Set to 10000 for sufficient exploration before exploitation begins
            epsilon = math.pow(epsilon_decay_value, episode - 10000) #Decreasing Epsilon value so that the model will exploit more

            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    # For checking mean reward after every 2000 episodes                
    total_reward += episode_reward 
    prior_reward = episode_reward # Save a record of the current episode reward to be used for comparison with future episode rewards

    if episode % 1000 == 0: 
        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

#print(q_table)
env.close()

For each episode, we initialize the variables and environment.

For each episode, while it is not done(out of bounds/pole tilted past angle) or truncated (episode length greater than 500), we generate a random value. If the value is greater than epsilon, the agent will exploit the values in the Q-table, otherwise it will explore and choose a random action.

After taking the action, we get the values of the new state and increment the episode reward by, then we discretize the new state.

If the new state is valid, we calculate the new Q-value for the state-action pair using the Bellman equation. Afterwards, we replace the old discrete state with the new one.

We set the minimum value of epsilon at 0.05, so that the agent will always be able to explore even at higher episodes and not drop close to 0 due to the decay. In order to decrement epsilon, we check 2 conditions: If the current episode’s reward is greater than the previous, meaning the model is improving and the agent should exploit more; If the episode is greater than 10000, to ensure the agent has explored sufficiently initially before beginning to exploit, to avoid convergence to a suboptimal policy.

Before the end of each episode, we save a record of the current episode’s reward so it can be compared against in the next episode.


# Reinforcment Learning agent based on the Q-values we obtained

In [ ]:
def Reinforcement_Learning_agent(state):
    discrete_state=get_discrete_state(state)
    action = np.argmax(q_table[discrete_state]) 
    return action

For Task 1, we can show the observation and chosen action below:

In [ ]:
observation = env.reset()
action = Reinforcement_Learning_agent(observation) #need to include [0] bcuz jupyter weird, colab noneed
print("Observation:", observation)
print("Chosen action:", action)

## Task 2: Demonstrate the effectiveness of the RL agent

For this task, use the agent developed in Task 1 to play the game for 100 episodes (refer to tutorial for how to play a round), record the cumulative reward for each round, and plot the reward for each round. A sample plotting code is given below. Note that you must include code to play for 100 episodes and use the code to obtain round_results for plotting. DO NOT record the round results in advance and paste the results to the notebook.

In [ ]:
env = gym.make("CartPole-v1")
test_episodes=100
episode_results = []

for test_episode in range(test_episodes):
    initial_state=env.reset()
    state=initial_state[0]
    cumulative_reward = 0
    done=False
    while not done:
        action = Reinforcement_Learning_agent(state)
        state, reward, done, info = env.step(action)
        cumulative_reward += reward
        #env.render()
    episode_results.append(cumulative_reward)
  #print("Cumulative reward for this round:", cumulative_reward)

In [ ]:
plt.plot(episode_results)
plt.title('Cumulative reward for each episode')
plt.ylabel('Cumulative reward')
plt.xlabel('episode')
plt.show()

In [ ]:
episode_results

Print the average reward over the 100 episodes.

In [ ]:
print("Average reward: ", sum(episode_results) / len(episode_results))
print("Is my agent good enough?", sum(episode_results) / len(episode_results) > 195)

## Task 3: Render one episode played by the agent

Plug your agent to the code below to obtain rendered result.

In [ ]:
!pip install moviepy

In [ ]:
env = RecordVideo(gym.make("CartPole-v1"), "./video")
print(env)
initial_observation = env.reset()
observation=initial_observation
i = 0
while True:
    i += 1
    env.render()
    #your agent goes here
    action = Reinforcement_Learning_agent(observation)
    observation, reward, done, info = env.step(action)
    if done: 
        break;    
env.close()
show_video()
if i==500:
  print("Im alive")
else:
  print("Die")

## Task 4: Format the Jupyter notebook by including step-by-step instruction and explanation